### Blending应用  
个人认为是将多个模型的处理结果组合到一起，然后建立组合特征与目标特征之间的模型。  
相比单一模型，组合特征后学习的模型效果更好。（视情况而定）

In [1]:
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedKFold,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

In [2]:
random.seed(78)
np.random.seed(78)

In [3]:
data=load_breast_cancer()
x=data['data']
y=data['target']

In [4]:
DT=DecisionTreeClassifier()
score=cross_val_score(DT,x,y,cv=5).mean()
score

0.9173730787144854

In [5]:
neighbor=KNeighborsClassifier()
score=cross_val_score(neighbor,x,y,cv=5).mean()
score

0.9279459711224964

In [6]:
clfs=[DecisionTreeClassifier(),
      KNeighborsClassifier()]

blending_train_x=np.zeros((x.shape[0],len(clfs)))
kf=StratifiedKFold(n_splits=5,shuffle=True,random_state=78)
for i,clf_ in enumerate(clfs):
    for train_index,test_index in kf.split(x,y):
        clf=clone(clf_)
        train_x,test_x=x[train_index],x[test_index]
        train_y,test_y=y[train_index],y[test_index]

        clf.fit(train_x,train_y)
        pred=clf.predict_proba(test_x)[:,1]  # 二分类问题，只取出为1的概率
        blending_train_x[test_index,i]=pred

clf=LogisticRegression()
clf.fit(blending_train_x,y)

clf.score(blending_train_x,y)

0.9420035149384886